# Set up notebook environment
## Note: This notebook should be run with a kernel for a conda environment with QIIME2 installed.

In [1]:
import os
import biom
import warnings
import pickle
import numpy as np
import pandas as pd
import qiime2 as q2
from biom import Table
from skbio import OrdinationResults
from skbio.stats import subsample_counts
from skbio.stats.distance import permanova, anosim, mantel
from skbio.stats.distance import DistanceMatrix
from qiime2.plugins.deicode.actions import rpca
from qiime2.plugins.feature_table.actions import rarefy
from qiime2.plugins.diversity.actions import beta_group_significance
from qiime2.plugins.emperor.actions import biplot, plot
from qiime2.plugins.diversity.actions import (beta,
                                              beta_phylogenetic,
                                              pcoa)
from qiime2.plugins import demux, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition

from assets.step_wise_anova import run_stepwise_anova
from qiime2.plugins.fragment_insertion.actions import filter_features
warnings.filterwarnings("ignore", category=DeprecationWarning)

# helper functions
from assets.util_updated import (mantel_matched, simulate_depth,
                        all_dists, all_dists_no_tree, nested_permanova)

# plotting
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
%matplotlib inline


# Stepwise ANOVA-RDA

## Run on data including a phylogeny
### Note: Replace the paths on lines 2-4 with those to your desired inputs

In [2]:
# Import data
metadata_q2 = q2.Metadata.load('metadata_samples.txt')
table_with_phylogeny = q2.Artifact.load('table_biom_with_phylogeny.qza')
phylogeny = q2.Artifact.load('phylogeny.qza')


### Note: Replace the value on line 15 below with the desired rarefaction depth

In [3]:
# Filter feature-table to include samples in metadata and re-index metadata file
table_with_phylogeny_biom = table_with_phylogeny.view(Table)
metadata_df = metadata_q2.to_dataframe()
shared_ = list(set(table_with_phylogeny_biom.ids()) & set(metadata_df.index))
metadata_df_shared_with_phylogeny = metadata_df.reindex(shared_)
table_with_phylogeny_biom_shared = table_with_phylogeny_biom.filter(shared_)
keep_ = table_with_phylogeny_biom_shared.ids('observation')[table_with_phylogeny_biom_shared.sum('observation') > 0]
table_with_phylogeny_biom_shared.filter(keep_, axis='observation')

# Import filtered table and re-indexed metadata file
table_with_phylogeny_biom_shared_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', table_with_phylogeny_biom_shared)
metadata_df_shared_with_phylogeny_q2 = q2.Metadata(metadata_df_shared_with_phylogeny)

# Generate distance matrices (using 'all_dists' function in assets/util.py)
rare_depth_with_phylogeny = 10000
dists_res_with_phylogeny = all_dists(table_with_phylogeny_biom_shared_q2,
                      rare_depth_with_phylogeny, phylogeny)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_with_phylogeny = {}
pcoa_res_with_phylogeny['Jaccard'] = pcoa(dists_res_with_phylogeny['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_with_phylogeny['Unweighted UniFrac'] = pcoa(dists_res_with_phylogeny['Unweighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_with_phylogeny['Weighted UniFrac'] = pcoa(dists_res_with_phylogeny['Weighted UniFrac'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_with_phylogeny['RPCA'] = dists_res_with_phylogeny['RPCA'].biplot.view(OrdinationResults).samples


### Note:  Replace the path on line 29 with that to where output should go

In [ ]:
# Perform stepwise ANOVA-RDA
es_all = {}
use_ = ['round','sample_type','sample_type_2','sample_type_3','biomass_sample','extraction_kit_round']

# Clean up metadata (only stuff to run)
mf_ord = metadata_df_shared_with_phylogeny_q2.to_dataframe().copy()

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]

# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_with_phylogeny.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_)

# Concatenate output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type_2':'Sample Type'}, axis=0)
es_alldf.to_csv('results/stepwise_anova/stepwise_anova_with_phylogeny.txt', sep='\t')
es_alldf


## Run on data without a phylogeny
### Note: Replace the paths on lines 2-3 with those to your desired inputs

In [17]:
# Import data
metadata = q2.Metadata.load('metadata_samples.txt')
table_without_phylogeny = q2.Artifact.load('table_without_phylogeny.qza')


### Note: Replace the value on line 15 below with the desired rarefaction depth

In [18]:
# Filter table
table_without_phylogeny_biom = table_without_phylogeny.view(Table)
metadata_df = metadata.to_dataframe()
shared_ = list(set(table_without_phylogeny_biom.ids()) & set(metadata_df.index))
metadata_df_shared_without_phylogeny = metadata_df.reindex(shared_)
table_without_phylogeny_biom_shared = table_without_phylogeny_biom.filter(shared_)
keep_ = table_without_phylogeny_biom_shared.ids('observation')[table_without_phylogeny_biom_shared.sum('observation') > 0]
table_without_phylogeny_biom_shared.filter(keep_, axis='observation')

# Import filtered table and re-indexed metadata file
table_without_phylogeny_biom_shared_q2 = q2.Artifact.import_data('FeatureTable[Frequency]', table_without_phylogeny_biom_shared)
metadata_df_shared_without_phylogeny_q2 = q2.Metadata(metadata_df_shared_without_phylogeny)

# Generate distance matrices (using 'all_dists_no_tree' function in assets/util.py)
rare_depth_without_phylogeny = 630
dists_res_without_phylogeny = all_dists_no_tree(table_without_phylogeny_biom_shared_q2,
                      rare_depth_without_phylogeny)

# Generate ordinations (row=samples, cols=axes)
pcoa_res_without_phylogeny = {}
pcoa_res_without_phylogeny['Jaccard'] = pcoa(dists_res_without_phylogeny['Jaccard'].distance_matrix).pcoa.view(OrdinationResults).samples
pcoa_res_without_phylogeny['RPCA'] = dists_res_without_phylogeny['RPCA'].biplot.view(OrdinationResults).samples


### Note:  Replace the path on line 29 with that to where output should go

In [ ]:
# Perform stepwise ANOVA-RDA
es_all = {}
use_ = ['round','sample_type','sample_type_2','sample_type_3','biomass_sample','extraction_kit_round']

# Clean up metadata (only stuff to run)
mf_ord = metadata_df_shared_without_phylogeny_q2.to_dataframe().copy()

# Filter data
keep_ = [v_ for v_ in mf_ord.columns
         if len(set(mf_ord[v_])) > 1 and
         len(set(mf_ord[v_])) < mf_ord.shape[0]//2]
mf_ord = mf_ord[keep_]

# Run stepwise ANOVA for all RDA ordinations
for metric_, ord_ in  pcoa_res_without_phylogeny.items():
    # get first three axes
    ord_ = ord_[[0,1,2]]
    ord_.columns = ['PC1','PC2','PC3']
    # subset/match
    mf_ord_ = mf_ord.copy()
    shared_ids = list(set(ord_.index)\
                      & set(mf_ord_.index))
    mf_ord_ = mf_ord_.loc[shared_ids,:]
    ord_ = ord_.loc[shared_ids,:]
    es_all[metric_] = run_stepwise_anova(ord_, mf_ord_, use_)

# Concatenate output from all runs and export
es_alldf = pd.concat(es_all).rename({'+ sample_type_2':'Sample Type'}, axis=0)
es_alldf.to_csv('results/stepwise_anova/stepwise_anova_without_phylogeny.txt', sep='\t')
es_alldf
